In [1]:
%load_ext autoreload
%autoreload complete

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from matplotlib_inline.backend_inline import set_matplotlib_formats
from tqdm.notebook import tqdm

from src.cache import cache

set_matplotlib_formats("svg")

In [2]:
from src.models.regression import get_lagged_df

df = get_lagged_df("media_print_protest", lags=range(-7, 1), ignore_group=True)
instruments = [c for c in df.columns if c.startswith("weather_") and c.endswith("lag0")]
treatment = "occ_protest_lag0"
outcome = "media_print_protest"

# normalize all instruments
for instrument in instruments:
    df[instrument] = (df[instrument] - df[instrument].mean()) / df[instrument].std()

# get covariances between instruments and treatment
covs = df[instruments + [treatment]].cov()
covs_w = covs.loc[instruments, treatment]

# get covariances between instruments and outcome
covs = df[instruments + [outcome]].cov()
covs_y = covs.loc[instruments, outcome]

covs = pd.concat([covs_w, covs_y], axis=1, keys=["cov_w", "cov_y"])
covs["wald"] = covs["cov_y"] / covs["cov_w"]
covs_cont = covs

import numpy as np
from sklearn.preprocessing import Binarizer


def discretize_optimally(X, y):
    best_threshold, best_cov = None, None
    for threshold in np.linspace(X.min(), X.max(), 1000):
        X_bin = Binarizer(threshold=threshold).fit_transform(X)
        cov = np.cov(X_bin.flatten(), y.flatten())[0, 1]
        if best_cov is None or abs(cov) > abs(best_cov):
            best_threshold = threshold
            best_cov = cov
    return Binarizer(threshold=best_threshold).fit_transform(X), best_threshold


bin_df = pd.DataFrame({treatment: df[treatment], outcome: df[outcome]})
# discretize all instruments optimally
for instrument in instruments:
    X = df[instrument].to_numpy().reshape(-1, 1)
    y = df[treatment].to_numpy().reshape(-1, 1)
    X_bin, threshold = discretize_optimally(X, y)
    print(f"{instrument}: {threshold:.3f}")
    bin_df[instrument] = X_bin.flatten()

# get covariances between instruments and treatment
covs = bin_df[instruments + [treatment]].cov()
covs_w = covs.loc[instruments, treatment]

# get covariances between instruments and outcome
covs = bin_df[instruments + [outcome]].cov()
covs_y = covs.loc[instruments, outcome]

covs = pd.concat([covs_w, covs_y], axis=1, keys=["cov_w", "cov_y"])
covs["wald"] = covs["cov_y"] / covs["cov_w"]
covs_bin = covs

covs = pd.concat([covs_cont, covs_bin], axis=1, keys=["cont", "bin"])

covs = covs.sort_values(ascending=False, key=abs, by=("cont", "cov_w"))
display(covs.head(20))

weather_tavg_lag0: -0.116
weather_tmin_lag0: 0.191
weather_tmax_lag0: -0.150
weather_prcp_lag0: -0.102
weather_snow_lag0: -0.149
weather_wspd_lag0: 0.489
weather_wpgt_lag0: 0.529
weather_pres_lag0: 0.805
weather_tsun_lag0: -1.016


cont                               bin            \
                      cov_w     cov_y          wald     cov_w     cov_y   
weather_tmin_lag0  0.007196  0.122098     16.966447  0.003568  0.044905   
weather_tavg_lag0  0.005044 -0.004635     -0.918894  0.003649 -0.009865   
weather_wspd_lag0 -0.004946 -0.475823     96.209843 -0.002560 -0.122810   
weather_snow_lag0 -0.004022 -0.197168     49.028452 -0.000988 -0.039882   
weather_wpgt_lag0 -0.003637 -0.373968    102.814500 -0.002579 -0.097116   
weather_tmax_lag0  0.003389 -0.065128    -19.218006  0.003038 -0.049028   
weather_pres_lag0 -0.002129  0.152524    -71.625450 -0.002306  0.034909   
weather_prcp_lag0  0.001964  0.148078     75.381395  0.001006  0.071345   
weather_tsun_lag0 -0.000008 -0.332245  44021.551793  0.001161  0.007952   

                              
                        wald  
weather_tmin_lag0  12.586974  
weather_tavg_lag0  -2.703669  
weather_wspd_lag0  47.965780  
weather_snow_lag0  40.356460  
weather_wpgt_lag0  37.658879  
weather_tmax_lag0 -16.136986  
weather_pres_lag0 -15.141292  
weather_prcp_lag0  70.894462  
weather_tsun_lag0   6.849487

In [3]:
confounders = [
    c
    for c in df.columns
    if c not in [outcome, treatment] + instruments and not c.startswith("weather_")
]
params = dict()
for instr in instruments:
    params[instr] = (
        sm.OLS(df[treatment], sm.add_constant(df[confounders + [instr]]))
        .fit()
        .params[instr]
    )
params_single = pd.DataFrame.from_dict(params, orient="index", columns=["coef"])
params_combi = pd.DataFrame(
    sm.OLS(df[treatment], sm.add_constant(df[confounders + instruments])).fit().params,
    columns=["coef"],
)
params_combi = params_combi[params_combi.index.str.startswith("weather_")]
params = pd.concat([params_single, params_combi], axis=1, keys=["single", "combi"])

params.sort_values(by=("single", "coef"), key=abs, ascending=False)

,single,combi
,coef,coef
weather_tmin_lag0,0.004243,0.010214
weather_tavg_lag0,0.003943,0.013798
weather_tmax_lag0,0.003469,-0.025601
weather_tsun_lag0,0.002878,0.009090
weather_wspd_lag0,-0.002499,-0.004411
weather_pres_lag0,-0.002007,-0.002958
weather_wpgt_lag0,-0.001789,0.000727
weather_snow_lag0,-0.001292,-0.000130
weather_prcp_lag0,0.000558,0.000748


In [4]:
# show that it does not help cv predictivity
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

classifiers = [
    LogisticRegression(max_iter=1000, solver="saga", class_weight="balanced"),
    RandomForestClassifier(n_estimators=100, class_weight="balanced"),
]
for cls in classifiers:
    print(cls)
    cvs1 = cross_val_score(cls, df[confounders], df[treatment], cv=5, scoring="f1")
    print(cvs1.mean(), cvs1.std())
    cvs2 = cross_val_score(
        cls, df[confounders + instruments], df[treatment], cv=5, scoring="f1"
    )
    print(cvs2.mean(), cvs2.std())
    # hypothesis test whether csv2 is better than cvs1
    print(stats.ttest_rel(cvs1, cvs2))

LogisticRegression(class_weight='balanced', max_iter=1000, solver='saga')


0.19291905036482265 0.09531609570718332


0.17154147443149256 0.06539172565713718
TtestResult(statistic=1.1177848954863863, pvalue=0.32625890974462823, df=4)
RandomForestClassifier(class_weight='balanced')
0.0010335917312661498 0.0020671834625322996
0.0011363636363636363 0.0022727272727272726
TtestResult(statistic=-0.9999999999999999, pvalue=0.373900966300059, df=4)


In [5]:
# season-independent params